# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [46]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [47]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 脫稿玩家, ID: 11445
Movie: 玩命關頭9, ID: 10482
Movie: 青春：第50屆日本全國高中棒球錦標賽, ID: 11450
Movie: 甲子園：夢想競技場, ID: 11449
Movie: 合法伴侶, ID: 11441
Movie: 雙面薇若妮卡 經典數位修復, ID: 11425
Movie: 人聲, ID: 11400
Movie: 薄荷糖, ID: 11368
Movie: 路卡的夏天, ID: 11268
Movie: 驅魔禁區, ID: 11439
Movie: 自殺突擊隊：集結, ID: 11328
Movie: 怪奇大廈, ID: 11438
Movie: 太陽出來之後, ID: 11435
Movie: 理髮師的男人 經典數位修復, ID: 11429
Movie: 破碎的瞬間, ID: 11421
Movie: 罪之聲, ID: 11420
Movie: 詭老, ID: 11409
Movie: 淘寶少年, ID: 11396
Movie: 小王子公主心, ID: 11389
Movie: LIP X LIP享受這世界的方法, ID: 11364
Movie: 叢林奇航, ID: 10315
Movie: 極速引爆, ID: 11428
Movie: 手捲煙, ID: 11424
Movie: 厲陰宅3：是惡魔逼我的, ID: 11376
Movie: 花束般的戀愛, ID: 11391
Movie: 貝加爾湖隱居札記：在這喧囂的世界，一個人到西伯利亞森林住半年, ID: 11375
Movie: 怪物奇兵 全新世代, ID: 11342
Movie: 韓國校園恐怖怪談：點名沒到的學生, ID: 11426
Movie: 野蠻糾碴隊, ID: 11300
Movie: 失控的審判, ID: 11201
Movie: 黑寡婦, ID: 10397
Movie: 死亡漩渦：奪魂鋸新遊戲, ID: 11311
Movie: 名偵探柯南：緋色的彈丸, ID: 11206
Movie: 阿依達的救援行動, ID: 11365
Movie: 一級任務, ID: 11346
Movie: 真．鮫島事件, ID: 11338
Movie: 永生戰, ID: 11073
Movie: 殺戮輪迴, ID: 11327


### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [48]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = movie_item[0]['value']

In [49]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
     'MV-Authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb'
}
    
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼
area_item = resp.json()

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in area_item:
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 彰化, 代號(area_id): 22
放映地區: 南投, 代號(area_id): 13
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14
放映地區: 基隆, 代號(area_id): 18
放映地區: 宜蘭, 代號(area_id): 11
放映地區: 花蓮, 代號(area_id): 12
放映地區: 台東, 代號(area_id): 9
放映地區: 金門, 代號(area_id): 24
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [50]:
# 指定放映地區
area_id = area_item[0]['area_id']

In [51]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

八月 12
八月 13
八月 14
八月 15
八月 16


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [52]:
# 選定要觀看的日期
date = "2021-08-12"

In [53]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [54]:
for result in html:
    print(f"電影院: {result['data-theater_name']}")
    movie_info = result.find_all("li", attrs={"class":"taps"})
    for j in movie_info:
        info = j.find_all('span')
        print(f"類型: {info[0].string}")
        
    movie_time = result.find_all("input")
    print('日期時間:')
    for time in movie_time:
        print('\t', time['value'])

電影院: 國賓影城(台北長春廣場)
類型: 數位
日期時間:
	 2021-08-12 13:30:00
	 2021-08-12 14:30:00
	 2021-08-12 15:50:00
	 2021-08-12 16:50:00
	 2021-08-12 18:10:00
	 2021-08-12 19:10:00
	 2021-08-12 20:30:00
	 2021-08-12 21:30:00
電影院: 欣欣秀泰影城
類型: 數位
日期時間:
	 2021-08-12 10:30:00
	 2021-08-12 11:40:00
	 2021-08-12 12:55:00
	 2021-08-12 14:05:00
	 2021-08-12 15:20:00
	 2021-08-12 16:35:00
	 2021-08-12 17:45:00
	 2021-08-12 19:10:00
	 2021-08-12 20:10:00
	 2021-08-12 21:35:00
	 2021-08-12 22:35:00
	 2021-08-13 00:00:00
電影院: 台北美麗華大直影城
類型: 數位
類型: IMAX
日期時間:
	 2021-08-12 19:10:00
	 2021-08-12 21:30:00
	 2021-08-12 23:50:00
	 2021-08-12 19:50:00
電影院: 華威天母影城
類型: 數位
日期時間:
	 2021-08-12 14:40:00
	 2021-08-12 17:00:00
	 2021-08-12 19:15:00
	 2021-08-12 21:30:00
電影院: 台北新光影城
類型: 數位
日期時間:
	 2021-08-12 20:15:00
	 2021-08-12 22:30:00
電影院: 台北in89豪華數位影城
類型: 數位
日期時間:
	 2021-08-12 18:30:00
	 2021-08-12 19:30:00
	 2021-08-12 21:00:00
	 2021-08-12 22:00:00
	 2021-08-12 23:30:00
電影院: 國賓大戲院(西門)
類型: 數位
日期時間:
	 2021-08-12 10:50:00
	 2021